In [2]:
# import all modules
import matplotlib.pyplot as plt
import numpy as np
import cv2
import csv
import os
import PIL
import tensorflow as tf
import pathlib
import pandas as pd
import gc

from PIL import Image

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from sklearn.model_selection import train_test_split

from keras import backend as K

In [4]:
# define image file paths
data_img_dir = '/kaggle/input/brain-anomaly-detection2/data/data/'
data_img_dir = pathlib.Path(data_img_dir)

# define training and validation data file paths
data_training = '/kaggle/input/brain-anomaly-detection2/data/train_labels.txt'
data_validation = '/kaggle/input/brain-anomaly-detection2/data/validation_labels.txt'

In [13]:
# load validation data; x is for images, y is for labels
validation_data_x, validation_data_y = [], []

# loop through all the images and labels - v2.0
with open(data_validation, 'r') as f:
    next(f)
    for line in f:
        img_id, label = line.strip().split(',')
        label = int(label)
        img_path = os.path.join(data_img_dir, img_id + '.png')
        img = cv2.imread(img_path)
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        validation_data_x.append(img_gray)
        validation_data_y.append(label)

# put images and labels into their respective arrays
validation_data_x = np.array(validation_data_x)
validation_data_y = np.array(validation_data_y)

In [7]:
# load training data; x is for images, y is for labels
training_data_x, training_data_y = [], []

# loop through all the images and labels - v3.0
with open(data_training, 'r') as f:
    next(f)
    for line in f:
        img_id, label = line.strip().split(',')
        label = int(label)
        img_path = os.path.join(data_img_dir, img_id + '.png')
        img = cv2.imread(img_path)
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        training_data_x.append(img_gray)
        training_data_y.append(label)

# put images and labels into their respective arrays
training_data_x = np.array(training_data_x, dtype='float16')
training_data_y = np.array(training_data_y, dtype='float16')

In [14]:
# necessary garbage collection (Kaggle has limited CPU)
gc.collect()

469

In [ ]:
# split data - deprecated
# training_data_X, validation_data_X, training_data_Y, validation_data_Y = train_test_split(training_data_x, training_data_y, random_state=0)

In [15]:
# get data shapes
print(training_data_x.shape)
print(validation_data_x.shape)

(15000, 224, 224)
(2000, 224, 224)


In [16]:
# set data shapes
training_data_x.shape = (15000, 224, 224, 1)
validation_data_x.shape = (2000, 224, 224, 1)

In [17]:
# define Mean F1-Score
@tf.function
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

@tf.function
def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

@tf.function
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [18]:
# necessary garbage collection (Kaggle has limited CPU)
gc.collect()

# define parameters for data augmentation
data_augmentation = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

# define the model
model = Sequential([
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.Conv2D(256, 3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid'),
])

# compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[f1_m, 'accuracy']
)

# define when the model will stop early
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='f1_m',
    mode='max',
    patience=10,
    verbose=1
)

# define learning rate
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=5,
    verbose=1
)

# train the model
model.fit(
    data_augmentation.flow(training_data_x, training_data_y, batch_size=32),
    epochs=100,
    steps_per_epoch=len(training_data_x) // 32,
    validation_data=(validation_data_x, validation_data_y),
    callbacks=[lr_scheduler, early_stopping]
)

Epoch 1/100
468/468 [==============================] - 82s 151ms/step - loss: 1.6843 - f1_m: 0.1739 - accuracy: 0.7150 - val_loss: 1.1888 - val_f1_m: 0.0233 - val_accuracy: 0.8620 - lr: 0.0010
Epoch 2/100
468/468 [==============================] - 67s 144ms/step - loss: 1.0711 - f1_m: 0.1254 - accuracy: 0.8310 - val_loss: 0.8805 - val_f1_m: 0.0185 - val_accuracy: 0.8625 - lr: 0.0010
Epoch 3/100
468/468 [==============================] - 67s 144ms/step - loss: 0.8610 - f1_m: 0.1661 - accuracy: 0.8429 - val_loss: 0.7489 - val_f1_m: 0.1005 - val_accuracy: 0.8680 - lr: 0.0010
Epoch 4/100
468/468 [==============================] - 68s 144ms/step - loss: 0.7659 - f1_m: 0.1910 - accuracy: 0.8503 - val_loss: 0.7987 - val_f1_m: 0.0132 - val_accuracy: 0.8625 - lr: 0.0010
Epoch 5/100
468/468 [==============================] - 68s 145ms/step - loss: 0.7371 - f1_m: 0.1948 - accuracy: 0.8516 - val_loss: 0.7268 - val_f1_m: 0.0079 - val_accuracy: 0.8625 - lr: 0.0010
Epoch 6/100
468/468 [==============

In [58]:
# necessary garbage collection (Kaggle has limited CPU)
gc.collect()

23

In [59]:
# evaluate the model
model.evaluate(validation_data_x,validation_data_y)

63/63 [==============================] - 1s 19ms/step - loss: 0.2672 - f1_m: 0.5000 - accuracy: 0.9085


[0.2671695947647095, 0.49998438358306885, 0.9085000157356262]

In [60]:
# necessary garbage collection (Kaggle has limited CPU)
gc.collect()

763

In [24]:
data_list = list(data_img_dir.glob('*'))
data_list.sort(key=lambda x : str(x))

testing_data_x = []

for i in range(17000, 22149):
    img = cv2.imread(str(data_list[i]))
    grayscaled_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    testing_data_x.append(grayscaled_img)

In [29]:
# necessary garbage collection (Kaggle has limited CPU)
gc.collect()

23

In [30]:
testing_data_x = np.array(testing_data_x, dtype='float16')
testing_data_x /= 255

In [32]:
# necessary garbage collection (Kaggle has limited CPU)
gc.collect()

23

In [33]:
# get testing data shape
print(testing_data_x.shape)

(5149, 224, 224)


In [38]:
# necessary garbage collection (Kaggle has limited CPU)
gc.collect()

69

In [36]:
# set testing data shape
testing_data_x.shape = (5149, 224, 224, 1)

In [62]:
prediction_data_x = model.predict(testing_data_x)

161/161 [==============================] - 3s 19ms/step


In [67]:
# necessary garbage collection (Kaggle has limited CPU)
gc.collect()

46

In [64]:
prediction_data_x_arr = []

for element in prediction_data_x:
    if element > 0.5:
        prediction_data_x_arr.append(1)
    else:
        prediction_data_x_arr.append(0)

In [48]:
# write results to .csv file
with open("/kaggle/working/result_labels1.csv", 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["id", "class"])
    for i in range(17001, 22150):
        writer.writerow(["0" + str(i), prediction_data_x_arr[i - 17001]])

In [ ]:
# necessary garbage collection (Kaggle has limited CPU)
gc.collect()